In [ ]:
!kaleido_get_chrome

In [ ]:
import sys

sys.path.append("..")

In [ ]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

pio.templates["plotly_white_custom"] = pio.templates["plotly_white"]

pio.templates["plotly_white_custom"].update({"layout_font_size": 52, "layout_font_family": "Times New Roman", "layout_font_color": "black"})

pio.templates.default = "plotly_white_custom"

labeled = True

In [ ]:
b = 5
for name in ["MNIST", "CIFAR10", "CH Reg"]:
    print(f"==== {name} ====")
    df = pd.read_pickle(f"../results/data_polys_{name}.pkl")

    not_finite = np.flatnonzero(~df["Finite"].fillna(True).astype(bool))
    df.iloc[not_finite, df.columns.get_loc("Volume")] = np.nan

    print(df.head())
    print("Rows:", len(df))
    print("Loaded Data")
    print("FINITE PERCENTAGES:")
    print(df[["Finite", "# Points"]].groupby("Finite", sort=False).sum() / df["# Points"].sum())

    print("Averaged volumes")
    print(df.groupby("Category")["Volume"].mean())

    gb = "Category"
    value_cols = ["Volume", "Log Volume", "Finite", "Inradius", "# Points"]
    hist = df.groupby(["# Faces", gb])[value_cols].agg({"Volume": "sum", "# Points": "sum", "Finite": "sum", "Inradius": "sum"})
    hist["# Polys"] = (
        df.groupby(["p", gb], sort=False).first().groupby(["# Faces", gb])[value_cols].size()
    )
    # hist = hist.join(df.groupby(["# Faces", gb], sort=False)[["# Points"]].sum())
    hist["Finite %"] = (100*hist["Finite"] / hist["# Polys"]).values.astype(int)
    hist["Average Volume"] = np.log(hist["Volume"] / hist["# Polys"])
    if "Inradius" in hist.columns: hist["Average Inradius"] = hist["Inradius"] / hist["# Polys"]
    hist["Log # Points"] = np.log(hist["# Points"])/np.log(b)
    # hist["% Correct"] = hist["# Correct"] / hist["# Points"]
    hist = hist.reset_index()
    hist.set_index(["# Faces", gb]).sort_index()

    
    print(df[df["# Points"] == 0]["# Faces"].mean())
    hist["Category"] = hist["Category"].astype(str)


    hist = hist.sort_values(["Category"], ascending=False)
    figs = dict()

    figs["FvP Finite"] = px.bar(
        hist,
        x="# Faces",
        y="# Polys",
        log_y=True,
        facet_row=gb,
        color="Finite %",
        width=1200,
        height=1200,
        hover_data=["# Points"],
    )

    figs["FvP Finite"].update_layout(legend_title_text="Finite Prop", showlegend=True, 
        coloraxis={
            "colorbar": {"title": {"text": "Percent<br>Finite<br><sup>&nbsp;</sup>", "side": "top"}, 
            "x":1.1,
            "ticksuffix": "%",
            "dtick": 20
            }, 
            "showscale": True,
            "cmin": 0,
            "cmax": 100}
    )
    figs["FvP Finite"].update_yaxes(dtick=1, type="log")

    if (hist["Average Volume"] < float("inf")).any():
        figs["FvP Volume"] = px.bar(
            hist,
            x="# Faces",
            y="# Polys",
            log_y=True,
            facet_row=gb,
            color="Average Volume",
            color_continuous_scale="Plasma",
            width=1200,
            height=1200,
            hover_data=["# Points"],
        )
        figs["FvP Volume"].update_layout(legend_title_text="Average Volume", showlegend=True,
            coloraxis={
                "colorbar": {"title": {"text": "Average<br>Volume<br><sup>&nbsp;</sup>", "side": "top"}, 
                "x":1.1,
                # "dtick": 1
                }, 
                "showscale": True}
        )

    figs["FvP Inradius"] = px.bar(
        hist,
        x="# Faces",
        y="# Polys",
        log_y=True,
        facet_row=gb,
        color="Average Inradius",
        color_continuous_scale="Plasma",
        width=1200,
        height=1200,
        hover_data=["# Points"],
    )
    figs["FvP Inradius"].update_layout(legend_title_text="Average Inradius", showlegend=True,
        coloraxis={
            "colorbar": {"title": {"text": "Average<br>Inradius<br><sup>&nbsp;</sup>", "side": "top"}, 
            "x":1.1,
            # "dtick": 1
            }, 
            "showscale": True}
    )

    figs["FvP Points Single"] = px.bar(
        hist,
        x="# Faces",
        y="# Polys",
        log_y=True,
        hover_data=[gb, "# Points"],
        color="Log # Points",
        width=1000,
        height=800,
        barmode="relative",
        opacity=1,
    )

    ticks = [b**i for i in range(int(np.ceil(np.log(hist["# Points"].max())/np.log(b))))]
    figs["FvP Points Single"].update_layout(legend_title_text="# Points", yaxis_title="Number of Polyhedrons" if labeled else None, xaxis_title="Number of Neighbors" if labeled else None, showlegend=labeled, 
        coloraxis={
            "colorbar": {"title": {"text": "Number<br>of Points<br><sup>&nbsp;</sup>", "side": "top"}, 
            # "xanchor":"right",
            "yanchor": "top", 
            "y":1.1, 
            # "x": 1,
            "len": 1.12,
            "tickvals": [np.log(tick)/np.log(b) for tick in ticks], "ticktext": ticks}, 
            "cmax": np.log(hist["# Points"].max())/np.log(b), 
            "cmin": 0,
            "showscale": labeled}
    )

    os.makedirs(f"../figures/{name}", exist_ok=True)
    for figname, fig in figs.items():
        fig.update_yaxes(dtick=1, type="log")
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
        path = f"../figures/{name}/{figname}_{name}"
        fig.write_image(path + ("_labeled" if labeled else "_unlabeled") + ".svg")
        print("Wrote image to", path + ("_labeled" if labeled else "_unlabeled") + ".svg")
        
    hist["Log # Polys"] = np.log(hist["# Polys"])


In [ ]:
df